In [6]:
import time
import numpy as np
import random
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## Read input data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [7]:
## Hyper parameters
NUM_MODELS = 4
EPOCHS = 1000
LEARNING_RATE = 0.1


In [8]:
## Construct model
class LearningModel:
    def __init__(self, sess):
        self.sess = sess
        self.build_model()
        
    def build_model(self):
        self.X = tf.placeholder(tf.float32, [None, 784])
        self.Y = tf.placeholder(tf.float32, [None, 10])

        W = tf.Variable(tf.zeros([784, 10]))
        b = tf.Variable(tf.zeros([10]))

        self.model = tf.matmul(self.X, W) + b

        # define cost and optimizer
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.model, labels=self.Y))
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE).minimize(self.cost)

        # define accuracy
        prediction = tf.equal(tf.argmax(self.model, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
        
    def do_predict(self, x_test):
        h = tf.nn.softmax(self.model)
        return self.sess.run(h, feed_dict={self.X:x_test})
    
    def get_accuracy(self, x_test, y_test):
        return self.sess.run(self.accuracy, feed_dict={self.X:x_test, self.Y:y_test})
    
    def do_train(self, x_test, y_test):
        return self.sess.run([self.optimizer, self.cost], feed_dict={self.X:x_test, self.Y:y_test})



In [9]:
## Main
models = []
model_input_size = int(mnist.train.images.shape[0]/NUM_MODELS)
with tf.Session() as sess:
    print("Create models for ensemble")
    for m in range(NUM_MODELS):
        models.append(LearningModel(sess))

    sess.run(tf.global_variables_initializer())

    ## Training
    t_start = time.time()
    avg_costs = np.zeros(len(models))
    for i in range(EPOCHS):
        for model_idx, model in enumerate(models):
            train_images = mnist.train.images[model_idx*model_input_size:(model_idx+1)*model_input_size, :]
            train_labels = mnist.train.labels[model_idx*model_input_size:(model_idx+1)*model_input_size, :]
            opt, cost_val = model.do_train(train_images, train_labels)
            avg_costs[model_idx] += cost_val/EPOCHS

    print('\nTraining time: %f secs' % (time.time() - t_start))
    print('Average Cost: ', avg_costs)
    print('\n')

    ## get ensemble accuracy
    test_size = len(mnist.test.labels)
    predictions = np.zeros([test_size, 10])
    for model_idx, model in enumerate(models):
        print("Model", model_idx, ' - Accuracy:', model.get_accuracy(mnist.test.images, mnist.test.labels))
        predictions += model.do_predict(mnist.test.images)
        ensemble_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))

    ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_prediction, tf.float32))
    print('Ensemble accuracy:', sess.run(ensemble_accuracy))


Create models for ensemble

Training time: 59.525995 secs
Average Cost:  [ 0.46117057  0.46054868  0.457278    0.43268508]


Model 0  - Accuracy: 0.9084
Model 1  - Accuracy: 0.9106
Model 2  - Accuracy: 0.9073
Model 3  - Accuracy: 0.9055
Ensemble accuracy: 0.9101
